In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import Union, Tuple
from types import NoneType
import random, shutil, os

In [2]:
def naive_csv_sampler(csv_path: str, sample_size: int, num_records: int | NoneType = None, header: str | NoneType = 'infer') -> pd.DataFrame:
    """Read samples of rows from csv file

    Args:
        csv_path (str): Path to file including file extensions
        sample_size (int): Number of rows to sample
        num_records (int | NoneType, optional): Total records in file, defaults to None. If None, the file will be scanned (costly)
        header (str | NoneType, optional): 'header'-parameter for pandas, defaults to 'infer'. Set to None if file has no header.

    Returns:
        pd.DataFrame: Dataframe with sampled entries (and potentially header)
    """
    if num_records is None:
        num_records = newlines_in_csv(csv_path)
    indices_skip = sorted(random.sample(range(1, num_records+1), num_records-sample_size))
    return pd.read_csv(csv_path, skiprows=indices_skip, header=header)

def newlines_in_csv(csv_path: str, chunk_size: int = 1024) -> int:
    """Counts number of newlines in csv file without loading entire file to memory.
    The number of newlines is the same as number of rows assuming,
        * EITHER csv has a header and last entry does not end with newline
        * OR csv does not have a header, but last entry ends with newline
        * ALWAYS data does not have any nested newline madness
    Originally from orlp, https://stackoverflow.com/a/64744699

    Args:
        csv_path (str): Path of csv file
        chunk_size (int, optional): How many KB to process at at a time. Defaults to 1024 = 1 MB.

    Returns:
        int: Number of newlines
    """
    chunk = chunk_size**2
    f = np.memmap(csv_path)
    number_newlines = sum(np.sum(f[i:i+chunk] == ord('\n')) for i in range(0, len(f), chunk))
    del f
    return number_newlines

def sample_all_hm(num_users: int) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Sample a dataset with `num_user` customers without any unreferenced rows.

    Args:
        num_users (int): How many users to sample from

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]: DataFrame of customers (1), transactions belonging to customers (2),
                                                            and articles belonging to transactions (3)
    """
    df_customers = naive_csv_sampler("dataset/customers.csv", num_users)
    df_transactions = pd.read_csv("dataset/transactions_train.csv") # TODO slow!!
    df_transactions = df_transactions[df_transactions['customer_id'].isin(df_customers['customer_id'])]
    
    df_articles = pd.read_csv("dataset/articles.csv")
    df_articles = df_articles[df_articles['article_id'].isin(df_transactions['article_id'])]
    return df_customers, df_transactions, df_articles

In [3]:
df_cust, df_tr, df_art = sample_all_hm(200)

df_cust.to_csv("dataset/customer_min.csv")
df_tr.to_csv("dataset/transactions_min.csv")
df_art.to_csv("dataset/articles_min.csv")

In [16]:
def copy_img_from_article(df: pd.DataFrame, outpath):
    for id in df['article_id']:
        id0 = "0" + str(id)
        img_path = f"./dataset/images/{id0[:3]}/{id0}.jpg"
        if not os.path.isfile(img_path):
            continue # ID has no image (happens for some cases)
        out_dir = f"./{outpath}/images/{id0[:3]}/"
        if not os.path.isdir(out_dir):
            os.makedirs(out_dir)
            shutil.copy(img_path, out_dir)
copy_img_from_article(df_art, "dataset_sample")

In [81]:
df = naive_csv_sampler("dataset/transactions_train.csv", sample_size=200)

In [82]:
df

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,aaf7a4cf881cc71b8cf97cd8e9c88ce300eb4fe2a279de...,649445003,0.059305,1
1,2018-09-21,31287b3d29b025cf00822b66b462a415e9c58d65385627...,620337036,0.016932,2
2,2018-09-23,04ebf0daa6de941f870109b5536bc226f574264bd13b25...,637673005,0.033881,2
3,2018-09-28,2e25374e1dd6141985ef534edabbe3ff436b395d1ce8d1...,672498003,0.025407,2
4,2018-10-12,93cb3a871d8997d85f8d765d37d5526b2eabab693919e4...,677219003,0.033881,2
...,...,...,...,...,...
195,2020-08-30,c6ae7c8e763d1127d6991e86a37d4e6fef69742ef2661c...,907527001,0.041441,2
196,2020-08-31,3296834ebcbd763dbd8d854f0883998bcf397cc02e6abb...,805947003,0.042356,2
197,2020-09-06,cfdc06ef05cf8e982bad3ce856bdcdbf4b141b35c2e1ad...,570189003,0.025407,2
198,2020-09-20,d4003b0349e30d5569547bb11ccd69669cdc9db6463c81...,715828028,0.033881,1


Number of rows in complete transactions csv:

In [33]:
newlines_in_csv("dataset/transactions_train.csv")

31788325

Trying out user-user collaborative filtering

In [19]:
"""Psudeocode
* Create customer profiles for all customers in (sampled) dataset
    * i.e. each customer ID has a vector r_ID whose elements represent items purchased
* Compute Jaccard similarity between all r_IDs, independent of position
* For a given customer x, choose the k customers closest to x
* For an article i, wether or not to recommend is based on the recommendation score
    r(x, i) = mean( [rel(y, i) for y in top k] )
"""
def position_indep_jaccard(x: list | set, y: list | set) -> float:
    # Position-independent jaccard-similarity
    x, y = set(x), set(y)
    try:
        return len(x.intersection(y)) / len(x.union(y))
    except ZeroDivisionError as e:
        print("Err this shouldn't happen")
        print(e, f"{x = }", f"{y = }", sep="\n")
        raise ZeroDivisionError

def find_customer_similarity(df_customer: pd.DataFrame, df_transactions: pd.DataFrame) -> Tuple[pd.DataFrame, dict]:
    articles_dict = {}
    for cust_ID in df_customer['customer_id']:
        articles_dict[cust_ID] = df_transactions['article_id'][df_transactions['customer_id'] == cust_ID].to_list()
        # Pop customers without purchase history
        if len(articles_dict[cust_ID]) == 0: articles_dict.pop(cust_ID)
    num_customers = len(df_customer)
    print(f"{num_customers = }")
    similarity_matrix = np.zeros((num_customers, num_customers))
    # Iterate over customers:
    for r, cust in enumerate(articles_dict.keys()):
        for c, second in enumerate(articles_dict.keys()):
            sim = position_indep_jaccard(articles_dict[cust], articles_dict[second])
            similarity_matrix[r,c] = sim
    
    return pd.DataFrame(similarity_matrix, index=articles_dict.keys(), columns=articles_dict.keys()), articles_dict

def get_recommendation(similarity_matrix: pd.DataFrame, articles_dict: dict, customer_ID: str, article_ID: int, k: int):
    # The k most similar customers IDs:
    closest_customers = similarity_matrix[customer_ID].sort_values(ascending=False)[:k].index
    return sum(1 if article_ID in articles_dict[cust] else 0 for cust in closest_customers)/k

sim_matr, art_dict = find_customer_similarity(df_cust, df_tr)


num_customers = 200


In [ ]:
import itertools
def get_n_recommendations(n: int, similarity_matrix: pd.DataFrame, articles_dict: dict, customer_ID: str, k: int, ignore_purchased: bool = True):
    # Get rec. score for all cases and choose n with highest score
    # ignore_purchased to ignore those articles customer has already bought
    blacklisted_articles = articles_dict[customer_ID] if ignore_purchased else []
    for art_ID in itertools.chain(*articles_dict.values()):
        score = get_recommendation(similarity_matrix, articles_dict, customer_ID, art_ID, k) 

In this case, two of the $k$ closest customers (including the customer itself) has bought the article in question. Thus we get a score of $\frac25=0.4$

In [26]:

get_recommendation(
    similarity_matrix=sim_matr,
    articles_dict=art_dict,
    customer_ID='008068b49b6bdd622ed406e30c8603270770174ebf300dbac0f5beac522921e0',
    article_ID=556255001,
    k=5
)


0.4

Methods for metric evaluation (MAP@12)

In [31]:
def prec(k: int, preds: np.ndarray, true: np.ndarray) -> float:
    """Precision function with cutoff (k). Used for MAP@12 metric.

    Args:
        k (int): Cutoff point for prediction array
        preds (np.ndarray): Prediction array
        true (np.ndarray): Ground truth

    Returns:
        float: Precision, i.e. portion of correctly predicted values

    """
    # Assumes that preds and true are 1d arrays ['a','b',...]
    return len(np.intersect1d(preds[:k], true))/k

def rel(k: int, preds: np.ndarray, true: np.ndarray) -> int:
    assert 0 < k <= len(preds), "k must be able to index preds!"
    return int(preds[k-1] in true)

def MAPk(k, preds, true) -> float:
    return np.mean([
        np.sum([prec(i,p,t)*rel(i,p,t) for i in range(1,k+1)])/\
            min(k, len(true))\
                for t, p in zip(true, preds)
    ])

In [29]:
# Tests
import unittest
class TestMetricFunctions(unittest.TestCase):
    def __init__(self, methodName: str = 'runTest') -> None:
        self.gt = np.array(['a', 'b', 'c', 'd', 'e'])
        self.preds1 = np.array(['b', 'c', 'a', 'd', 'e'])
        self.preds2 = np.array(['a', 'b', 'c', 'd', 'e'])
        self.preds3 = np.array(['f', 'b', 'c', 'd', 'e'])
        self.preds4 = np.array(['a', 'f', 'e', 'g', 'b'])
        self.preds5 = np.array(['a', 'f', 'c', 'g', 'b'])
        self.preds6 = np.array(['d', 'c', 'b', 'a', 'e'])
        super().__init__(methodName)

    def test_prec(self):
        self.assertAlmostEqual(prec(1, self.preds1, self.gt), 1.0)
        self.assertAlmostEqual(prec(1, self.preds2, self.gt), 1.0)
        self.assertAlmostEqual(prec(1, self.preds3, self.gt), 0.0)
        self.assertAlmostEqual(prec(2, self.preds4, self.gt), 0.5)
        self.assertAlmostEqual(prec(3, self.preds5, self.gt), 2/3)
        self.assertAlmostEqual(prec(3, self.preds6, self.gt), 1.0)
    
    def test_rel(self):
        self.assertAlmostEqual(rel(1, self.preds1, self.gt), 1.0)
        self.assertAlmostEqual(rel(1, self.preds2, self.gt), 1.0)
        self.assertAlmostEqual(rel(1, self.preds3, self.gt), 0.0)
        self.assertAlmostEqual(rel(2, self.preds4, self.gt), 0.0)
        self.assertAlmostEqual(rel(3, self.preds5, self.gt), 1.0)
        self.assertAlmostEqual(rel(3, self.preds6, self.gt), 1.0)
    
    def test_mapk(self):
        all_true = np.array([self.gt for i in range(6)])
        all_pred = np.array([self.preds1, self.preds2, self.preds3,\
                            self.preds4, self.preds5, self.preds6])
        self.assertAlmostEqual(MAPk(k=4, preds=all_pred, true=all_true), 0.71875)
unittest.main(argv=[''], verbosity=2, exit=False)

test_mapk (__main__.TestMetricFunctions) ... ok
test_prec (__main__.TestMetricFunctions) ... ok
test_rel (__main__.TestMetricFunctions) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK
